In [273]:
import urllib.request
import requests
from IPython.display import display, Math, Latex
from tqdm import tqdm
import seaborn as sns
import geopandas as gpd
import math
import time as ti
import json
import os
import sys

import branca.colormap as cm
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point, Polygon

sys.path.insert(0, "../src")


pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)


# Now let's try to compute the distance OD Matrix between these points

First, let's define the function need to perform the HTTP GET request. All the assumptions regarding this matrix are under this first link below.

- Resources: https://docs.microsoft.com/en-us/bingmaps/rest-services/examples/distance-matrix-asynchronous-example

- Example: https://docs.microsoft.com/en-us/bingmaps/rest-services/examples/distance-matrix-example


The HTTP GET request works as follow:

https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins={lat0,long0;lat1,lon1;latM,lonM}&destinations={lat0,lon0;lat1,lon1;latN,longN}&travelMode={travelMode}&startTime={startTime}&timeUnit={timeUnit}&key={BingMapsKey}

- origins: list of lat, long coordinates of origins
- destionations: list of lat, long coordinates of destinations
- travelMode: `Driving`
- startTime: `2021-11-15-07:00`
- timeUnit
- BingMapsKey


In [274]:
def get_data(orig, dest, size_x, size_y):
    # Your Bing Maps Key
    bingMapsKey = "Ah52Budr_8V_sE1HKdPH-INIK_RDqas1zgkSJwdtB3GW3Dm04DsNm3lzkoXmpPc9"

    # input information
    travelMode = "Driving"
    startTime = "2021-11-15-07:00"
    distanceUnit = "miles"
    timeUnit = "minute"

    routeUrl = (
        "https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?"
        + f"origins={orig}"
        + f"&destinations={dest}"
        + f"&travelMode={travelMode}"
        + f"&startTime={startTime}"
        + f"&timeUnit={timeUnit}"
        + f"&key={bingMapsKey}"
    )

    # request = urllib.request.Request(routeUrl)
    # response = urllib.request.urlopen(request, proxies={"http":"http://myproxy:3129"})
    # data = json.load(response)

    response = requests.get(routeUrl)
    data = response.json()

    distance, time = build_od_matrix(data, (size_x, size_y))
    return distance, time

In [275]:
def build_od_matrix(data, size):

    resources = data["resourceSets"][0]["resources"][0]["results"]

    distance = np.zeros((size[0], size[1]))
    time = np.zeros((size[0], size[1]))

    for i in range(size[0]):
        for j in range(size[1]):
            distance[i, j] = resources[i * size[1] + j]["travelDistance"]
            time[i, j] = resources[i * size[1] + j]["travelDuration"]
            
    return distance, time

# Loading the data


In [276]:
# load the housing data
residences = pd.read_csv("../data/HousingData.csv")
residences = residences[residences["Residence"] != 'Simmons Hall']
residences.loc[len(residences.index)] = ["Depot", "139 Massachusetts Ave, Cambridge, MA 02138", 42.36427095750911, -71.10169629169343, "NA", "NA"]

mit_buildings = pd.read_csv("../data/MIT_Buildings.csv")
shuttle_stops = pd.read_csv("../data/current_stops.csv").rename(
    columns={"Station Name": "Name"}
)

origins = residences[["Residence", "Long", "Lat"]].rename(
    columns={"Residence": "Name"})

# add a row to origins
destinations = shuttle_stops.append(
    residences[["Residence", "Lat", "Long"]].rename(
        columns={"Residence": "Name"})
)
destinations = destinations.append(mit_buildings[["Name", "Lat", "Long"]])
# destinations.loc[len(destinations.index)] = ["Depot", 42.36381606868144, -71.0885840857672]

# save origins and destionations to CSV
origins.to_csv("../data/origins.csv", index=False)
destinations.to_csv("../data/destinations.csv", index=False)

/var/folders/hk/nxk7bnsj4kq9nwsmnmdtzhb80000gn/T/ipykernel_58158/298089451.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  destinations = shuttle_stops.append(
/var/folders/hk/nxk7bnsj4kq9nwsmnmdtzhb80000gn/T/ipykernel_58158/298089451.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  destinations = destinations.append(mit_buildings[["Name", "Lat", "Long"]])


In [277]:
N_o = origins.shape[0]
N_d = destinations.shape[0]

# Matrix Initialization
distance_matrix = np.zeros((N_o, N_d))
time_matrix = np.zeros((N_o, N_d))


In [278]:
# Compute the list of origin-destinations points
long = list(origins["Long"])
lat = list(origins["Lat"])

or_points = []
for i, j in zip(lat, long):
    or_points.append(str(i) + "," + str(j))

# Compute the list of origin-destinations points
long = list(destinations["Long"])
lat = list(destinations["Lat"])

des_points = []
for i, j in zip(lat, long):
    des_points.append(str(i) + "," + str(j))

or_points = ";".join(or_points)
des_points = ";".join(des_points)

In [279]:
distance_matrix, time_matrix = get_data(
    or_points, des_points, size_x=int(N_o), size_y=int(N_d)
)

In [280]:
distances = pd.DataFrame(distance_matrix)

# give a destination to each column name
distances.columns = destinations["Name"]
distances.index = origins["Name"]

# Reshape dataframe to long dataframe
distances = distances.reset_index()
distances = pd.melt(
    distances,
    id_vars=["Name"],
    value_vars=distances.columns.values.tolist()[1:],
    var_name="Destination",
    value_name="Distance",
)

In [281]:
times = pd.DataFrame(time_matrix)

# give a destination to each column name
times.columns = destinations["Name"]
times.index = origins["Name"]

# Reshape dataframe to long dataframe
times = times.reset_index()
times.to_csv("../data/origin_destination_times.csv", index=False)
times = pd.melt(
    times,
    id_vars=["Name"],
    value_vars=times.columns.values.tolist()[1:],
    var_name="Destination",
    value_name="Time (min)",
)

In [282]:
distances["Time (min)"] = times["Time (min)"]
distances = distances.rename(columns={"Name": "Origin"})
distances.head()


,Origin,Destination,Distance,Time (min)
0,Baker House,Kendall Square T,2.752,7.6167
1,Burton Conner,Kendall Square T,2.578,8.0167
2,East Campus,Kendall Square T,0.520,1.8500
3,MacGregor House,Kendall Square T,2.497,7.9667
4,Maseeh Hall,Kendall Square T,1.124,3.9333


In [283]:
distances.to_csv("../data/origin_destination_distances.csv", index=False)

In [284]:
# show more rows
pd.set_option("display.max_rows", 500)

times

,Name,Destination,Time (min)
0,Baker House,Kendall Square T,7.6167
1,Burton Conner,Kendall Square T,8.0167
2,East Campus,Kendall Square T,1.8500
3,MacGregor House,Kendall Square T,7.9667
4,Maseeh Hall,Kendall Square T,3.9333
...,...,...,...
850,Sidney-Pacific,MIT Plasma Science and Fusion Center,1.7333
851,Tang Hall,MIT Plasma Science and Fusion Center,4.8167
852,The Warehouse,MIT Plasma Science and Fusion Center,1.3000
853,Westgate Apartments,MIT Plasma Science and Fusion Center,4.1167


In [285]:
times = pd.DataFrame(time_matrix)
times.columns = destinations["Name"]
times.index = origins["Name"]

# Reshape dataframe to long dataframe
times = times.reset_index()


In [286]:
times

Name,Name,Kendall Square T,Wadsworth at Amherst,Media Lab,Media Lab at Ames,Amherst at Kresge,Burton/MacGregor,Tang/Westgate,W98 at Vassar St,W92 at Amesbury St,Simmons Hall,New Vassar Dorm,Vassar St at Mass Ave,Main at Vassar,Baker House,Burton Conner,East Campus,MacGregor House,Maseeh Hall,McCormick Hall,New House,Next House,New Vassar,Random Hall,70 Amherst Street,Ashdown House,Edgerton House,Graduate Tower,Sidney-Pacific,Tang Hall,The Warehouse,Westgate Apartments,Depot,MIT Chapel,Hyatt Regency Cambridge,MIT Furniture Exchange,Veterans Memorial Pool,Great Dome,MIT Sloan School of Management,MIT Medical Urgent Care Service,Walker Memorial,MIT Department of Mathematics,MIT Sailing Pavilion,The Muddy Charles Pub,Tudor Dog Park,MIT Plasma Science and Fusion Center
0,Baker House,7.6167,7.7833,8.5167,8.3500,6.8000,3.2833,4.2667,3.3333,3.4833,3.9000,4.5833,5.7500,6.9000,0.0000,1.5167,8.6167,3.2833,6.5833,7.0667,3.2833,4.2667,8.1500,7.0667,8.5167,6.1333,6.8667,7.9833,6.7500,3.0333,6.3833,3.8667,7.8167,7.3833,3.3667,4.1833,5.0333,7.6500,8.4167,10.1500,8.6167,6.9167,6.7833,8.3500,6.1833,7.1833
1,Burton Conner,8.0167,8.1833,8.9167,8.7667,7.2000,1.7667,2.7333,3.7500,3.8833,4.3000,4.9833,6.1667,7.3000,9.2333,0.0000,9.0167,1.7667,7.0000,7.4833,1.7667,2.7333,8.5667,7.4667,8.9167,6.5333,7.2667,8.3833,7.1500,3.4333,6.8000,4.2833,8.2333,7.7833,3.7667,4.5833,5.4333,8.0667,8.8167,10.5667,9.0167,7.3167,7.2000,8.7667,6.5833,7.5833
2,East Campus,1.8500,1.8500,0.6000,0.7667,2.9833,3.2833,5.1667,4.1167,4.2500,4.7333,4.3667,3.7833,4.9000,3.4333,3.0667,0.0000,3.2833,2.7833,2.9167,3.2833,4.2500,7.5500,4.9000,0.6000,5.6333,4.8000,2.2167,6.1667,3.8167,5.7833,4.6500,6.3833,3.5667,4.1333,4.9833,5.8333,4.7500,3.3333,2.1500,0.2333,1.2333,4.4833,0.7667,7.2500,5.0167
3,MacGregor House,7.9667,8.1333,8.8667,8.7000,7.1500,0.1167,0.9667,3.7000,3.8333,4.2500,4.9333,6.1167,7.2500,9.1667,8.7833,8.9667,0.0000,6.9500,7.4333,1.6000,0.9667,8.5000,7.4167,8.8667,6.4833,7.2167,8.3333,7.1000,3.3833,6.7500,0.8167,8.1667,7.7333,3.7167,4.5333,5.3833,8.0000,8.7667,10.5000,8.9667,7.2667,7.1500,8.7000,6.5333,7.5333
4,Maseeh Hall,3.9333,3.2333,3.9667,4.2000,0.2667,2.6500,4.5500,3.5000,3.6333,3.0000,2.6333,2.0500,3.2333,2.2333,2.2667,4.6000,2.6500,0.0000,0.5333,2.6500,3.6333,5.8167,3.1667,3.9667,3.9000,3.0667,4.3167,4.4333,3.1833,4.0500,4.0333,4.6500,1.0167,3.5167,4.3500,5.2167,3.0167,3.8667,5.6167,4.6000,2.4167,2.3000,4.2000,5.5167,3.2833
5,McCormick Hall,4.5333,3.8333,4.5667,4.8000,0.4167,2.9833,4.1333,3.0833,3.2167,3.6000,3.2333,2.6500,3.8333,1.6833,1.7167,5.1833,2.9833,0.6500,0.0000,2.9833,3.9667,6.4167,3.7667,4.5667,4.5000,3.6667,4.9167,5.0333,2.7833,4.6500,3.6167,5.2500,1.7833,3.1000,3.9167,4.7667,3.6167,4.4667,6.2167,5.1833,3.0167,2.9000,4.8000,6.1167,3.8833
6,New House,5.3667,6.4333,6.0500,6.1167,4.5500,1.7333,0.9667,3.7000,2.0833,1.8167,2.3333,3.5167,4.6667,6.5833,6.1833,6.3667,1.6000,4.3500,4.8333,0.0000,0.9667,8.5000,4.8333,6.0500,6.4833,4.7333,5.7500,7.1000,3.3833,6.7500,0.8167,6.3167,5.1500,3.7167,4.5333,5.3833,5.4167,6.8500,6.8167,6.3667,7.2667,7.1500,6.1167,6.5333,4.9500
7,Next House,4.8833,5.9500,5.5667,5.6333,4.0667,1.9000,0.8000,1.8667,1.6000,1.3333,1.8500,3.0333,4.1833,6.1000,5.7000,5.8833,1.9000,3.8667,4.3500,1.9000,0.0000,7.0000,4.3500,5.5667,5.0833,4.2500,5.2667,5.6167,2.1000,5.2333,0.3333,5.8333,4.6667,2.3667,3.8167,4.6667,4.9333,6.3667,6.3333,5.8833,5.3333,5.2167,5.6333,5.8000,4.4667
8,New Vassar,4.8667,5.9167,5.5167,5.5833,4.2833,6.1167,5.6500,5.3333,5.0833,3.9667,3.6000,3.5167,4.0833,6.3167,5.9000,5.8500,6.1167,4.0833,4.5667,6.1167,5.6500,0.0000,3.4000,5.5167,1.5000,1.5667,5.2333,1.6833,5.5833,0.5500,5.1333,4.4167,4.8833,5.8500,5.1833,7.4167,5.1500,6.3500,6.3167,5.8500,5.8833,5.7667,5.5833,2.7667,1.8667
9,Random Hall,3.8833,4.9333,4.5333,4.6000,3.0333,4.8667,4.3833,4.0833,3.8333,2.7167,2.3500,2.2667,3.0833,5.0500,4.6500,4.8500,4.8667,2.8333,3.3167,4.8667,4.3833,4.3000,0.0000,4.5333,2.3833,1.5500,4.2500,1.9833,4.3167,2.5333,3.8667,1.9500,3.6167,4.5833,5.31